In [1]:
from openpyxl import load_workbook

In [2]:
import yaml
import re

In [3]:
example_yaml = '/home/bay001/projects/codebase/cellranger_cwl/development/jobs/cellranger_workflow.yaml'
example_workbook = '/home/bay001/projects/codebase/cellranger_cwl/testdata/template_sequencing_runs_manifest_20170502.xlsx'

In [4]:
f = yaml.load(file(example_yaml, 'r'))

In [5]:
wb = load_workbook(
    filename = example_workbook
)

In [6]:
sheet = wb['Sample Information']

In [7]:
print(sheet['A24'].value)

None


In [8]:
igm_expt_mapper = {
    'Contact Name':'investigator',
    'Date of Sample Submission':'study_date',
    'PI Name':'pi_name',
    'Contact Email':'contact_email',
    'Institute/Company Name':'institute',
    'Index number or PO':'po_number',
    'Member of MCC':'member_of_MCC',
    'Member of DRC':'member_of_DRC',
    'Number of cycles':'number_of_cycles',
    'Type of Run':'type_of_run',
    'PI Email':'pi_email'
}
igm_sample_mapper = {
    'Sample Name':'sample_names',
}
igm_sample_metadata_mapper = {
    'quantification':'Quantification Method',
    'library_prep':'Library Prep Method',
    'uid':'Sample Code',
    'size':'Library Size (bp)',
    'volume':'Volume (ml)',
    'concentration':'Conc. (nM)'
}

In [9]:
def get_max_lines(sheet):
    """
    returns the max dimension (either row or col) of a sheet.
    """
    dim = sheet.calculate_dimension()
    return max(
        [int(i) for i in re.findall('[\d]+',dim)]
    )

In [10]:
sheet

<Worksheet "Sample Information">

In [11]:
def get_tofill(sheet, label, row_start=1, col_start=1, horizontal=True, offset=1):
    """
    returns the cell that needs to be filled given a field label.
    params:
    
    sheet: openpyxl.Worksheet
    label: Worksheet cell value
    row_start: int
        topmost value in workbook (1 = 1)
    col_start: int
        leftmost value in workbook (A = 1)
    horizontal: 
        if True, return position of label + offset (to the right, horizontally)
        if False, return position of label + offset (downward, vertically)
    offset: int
    """
    for i in range(row_start, get_max_lines(sheet)+1):
        for j in range(col_start, get_max_lines(sheet)+1):
            field = sheet.cell(row=i, column=j).value
            if label == field:
                if horizontal:
                    return i, (j+offset)
                else:
                    return (i+offset), j
    print("found nothing")

In [12]:
def fill_expt_details(sheet, igm_expt_mapper):
    for igm_label, yml_label in igm_expt_mapper.iteritems():
        # print(field, value)
        if yml_label in f.keys():
            print(igm_label)
            row, col = get_tofill(sheet, igm_label)
            sheet.cell(row=row, column=col).value = f[yml_label]
            
def fill_sample_details(sheet, igm_sample_mapper, igm_sample_metadata_mapper):
    
    for igm_label, yml_label in igm_sample_mapper.iteritems():
        if yml_label in f.keys():
            offset = 0
            for sample in f[yml_label]:
                
                for sample_id, sample_metadata in sample.iteritems():
                    offset += 1
                    row, col = get_tofill(sheet, igm_label, offset=offset, horizontal=False)
                    sheet.cell(row=row, column=col).value = sample_id
                    for metakey, metadata in sample_metadata.iteritems():
                        label_row, label_col = get_tofill(sheet, igm_sample_metadata_mapper[metakey], offset=0, horizontal=True)
                        row, col = get_tofill(sheet, igm_sample_metadata_mapper[metakey], offset=offset, horizontal=False)
                        sheet.cell(row=row, column=label_col).value = metadata

In [13]:
fill_expt_details(sheet, igm_expt_mapper)
fill_sample_details(sheet, igm_sample_mapper, igm_sample_metadata_mapper)
wb.save('/home/bay001/projects/codebase/cellranger_cwl/testdata/filled_sequencing_runs_manifest_20170502.xlsx')

Date of Sample Submission
PI Email
Index number or PO
PI Name
Member of MCC
Contact Email
Type of Run
Contact Name
Institute/Company Name
Member of DRC
Number of cycles
